- 자연어 처리(NLP)  : 인간의 언어를 컴퓨터가 이해하고 처리 할수 있도록 하는 인공지능
- 주요 요소
  - 토큰화(Tokenization) : 문장을 단어 또는 문장단위로 나누는 과정
  - 품사 태깅(POS Tagging) : 각 단어의 품사(명사,동사)를 분석
  - 어휘 및 의미 분석(Semantic Analysis) : 단어간의 관계를 분석 문맥을 파악
  - 구분 분석(Syntactic Parsing): 문장의 문법적 구조를분석
  - 감성 분석(Sentiment Analysis) : 텍스트에서 감정을 분석(긍정, 부정)

- NLP 활용
  - 챗봇
    - 고객 서비스 자동화
    - 의료 상담 보조
    - 금융 상담 보조
  - 기계 번역
    - 구글번역
    - 네이버 파파고
    - 딥러닝 기반 번역 서비스
  - 텍스트 분석
    - 감성 분석
    - 자동 요약(뉴스요약)
    - 키워드 추출 및 정보 검색

In [1]:
!pip install transformers

감성분석

In [10]:
from transformers import pipeline
# 감성 분석 모델 로드
classifier = pipeline("sentiment-analysis")
# 예제 텍스트 분석
text = ["I love this movie!","This product is terrible..."]
result = classifier(text)

for t, r in zip(text,result):
  print(f"Text : {t}=> sentiment: {r['label']}  :  {r['score']:.2f}")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Text : I love this movie!=> sentiment: POSITIVE  :  1.00
Text : This product is terrible...=> sentiment: NEGATIVE  :  1.00


기계번역

In [13]:
from transformers import pipeline
# 번역모델 로드
translator = pipeline(model = "Helsinki-NLP/opus-mt-ko-en")
# 번역할 문장
text = "안녕하세요, 반갑습니다."
# 번역 실행
translation = translator(text)
print(f"original : {text}\ntranslated : {translation}")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


original : 안녕하세요, 반갑습니다.
translated : [{'translation_text': 'Hello. Nice to meet you.'}]


질문 답변 시스템

In [14]:
from transformers import pipeline
# 질문 답변 모델 로드
qa_pipeline = pipeline("question-answering")
# 문맥과 질문 입력
context = "Albert Einstein was a theoretical physicist who developed the theory of relativity."
quest = "Who developed the theory of relativity?"
# 답변 생성
answer = qa_pipeline(question=quest, context=context)
# 결과 출력
print(f"Question: {quest}\nAnswer: {answer['answer']}")

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Question: Who developed the theory of relativity?
Answer: Albert Einstein


In [27]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Base Model (108M)
tokenizer = AutoTokenizer.from_pretrained("Dilwolf/Kakao_app-kr_sentiment")
model = AutoModelForSequenceClassification.from_pretrained("Dilwolf/Kakao_app-kr_sentiment",num_labels=2)

# 분석할  한글 문장
texts = ['이 영화는 정말 재미있어요','너무 지루하고 시간 낭비였어요','완전 감동적이고 최고였어요','이 가게 서비스가 최악이에요']

def sentiment_analysis(text):
  inputs = tokenizer(text,return_tensors='pt',truncation=True, padding=True, max_length=512)
  #  모델 예측 수행
  with torch.no_grad():
    outputs = model(**inputs)

  logits = outputs.logits  # [1,2] 크기의 tensor(부정 긍정)
  probabilities= torch.nn.functional.softmax(logits, dim=-1)
  labels = ['부정','긍정']
  predicted_label =  labels[torch.argmax(probabilities).item()]  # 예측된 라벨
  confidence = torch.max(probabilities).item()  # 확신도

  return predicted_label  , confidence
# 추론
for text in texts:
    predicted, conf = sentiment_analysis(text)
    print(f"원본:{text} 감정:{predicted} 확률:{conf}")

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

원본:이 영화는 정말 재미있어요 감정:긍정 확률:0.9978325963020325
원본:너무 지루하고 시간 낭비였어요 감정:부정 확률:0.9367395043373108
원본:완전 감동적이고 최고였어요 감정:긍정 확률:0.9977447986602783
원본:이 가게 서비스가 최악이에요 감정:부정 확률:0.9988149404525757


In [29]:
text = '토요일 웨이팅 각오하고 방문했는데, 저녁 6시 정도에 30분 대기하고 들어갔어요! 한판수육전골, 미나리오징어전, 매콤스지볶음 시켜먹었는데 한판수육전골이 제일 맛있고 육수도 계속 추가할 수 있어서 좋았습니다. 미나리오징어전은 건새우맛이 많이나서 제 취향은 아니였고 매콤스지볶음은 불닭볶음면 정도로 약간 많이 매워서 호불호 있을 듯 싶어요!! 그래도 아재감성 분위기에 수육전골이 소주를 부르는 맛이어서 수육전골만 다시 먹거나 곱창전골 도전하러 재방문 할 것 같아요'
predicted, conf = sentiment_analysis(text)
print(f"원본:{text} \n감정:{predicted} \n확률:{conf}")

원본:토요일 웨이팅 각오하고 방문했는데, 저녁 6시 정도에 30분 대기하고 들어갔어요! 한판수육전골, 미나리오징어전, 매콤스지볶음 시켜먹었는데 한판수육전골이 제일 맛있고 육수도 계속 추가할 수 있어서 좋았습니다. 미나리오징어전은 건새우맛이 많이나서 제 취향은 아니였고 매콤스지볶음은 불닭볶음면 정도로 약간 많이 매워서 호불호 있을 듯 싶어요!! 그래도 아재감성 분위기에 수육전골이 소주를 부르는 맛이어서 수육전골만 다시 먹거나 곱창전골 도전하러 재방문 할 것 같아요 
감정:긍정 
확률:0.9437845349311829


In [33]:
text = '''일요일 저녁에 돼지갈비에 탄수화물 먹고싶어서 방문했어요.
결국 못참고 진로 골드 순삭할뻔 했네요
두번 갈 곳이 못됩니다.
'''
predicted, conf = sentiment_analysis(text)
print(f"원본:{text} \n감정:{predicted} \n확률:{conf}")

원본:일요일 저녁에 돼지갈비에 탄수화물 먹고싶어서 방문했어요.
결국 못참고 진로 골드 순삭할뻔 했네요
두번 갈 곳이 못됩니다.
 
감정:부정 
확률:0.5299375653266907


# 한글-영어 번역

In [36]:
from transformers import pipeline
def transKoEng(text):
  # 번역모델 로드
  translator = pipeline(model = "Helsinki-NLP/opus-mt-ko-en")
  # 번역 실행
  translation = translator(text)
  return translation

# print(f"original : {text}\ntranslated : {translation}")

In [37]:
text = '파이낸셜타임스(FT)는 10일(현지시간) 중국 컨설팅업체 첸잔 자료를 인용해 900억 달러(약 130조원) 규모인 D램 시장에서 CXMT 점유율이 2020년만 해도 제로에 가까웠지만 지난해 5%로 늘어났다고 보도했다.'
transKoEng(text)

Device set to use cpu


[{'translation_text': "The Financial Times reported that in 10 days, China's consulting firm Chenzan data, the CXMT share was close to zero in 2020, but increased by 5% last year."}]

질문 답변 한글모델

In [58]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
# Base Model (108M)
model_name = 'monologg/koelectra-base-v3-finetuned-korquad'
# 토크나이져 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
# 모델 로드
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# 질문-답변 함수 정의
def answer_question(question,context):
  inputs = tokenizer(question,context,return_tensors='pt',truncation=True, padding=True, max_length=512)
  #  모델 예측 수행
  with torch.no_grad():
    outputs = model(**inputs)
  start_scores = outputs.start_logits
  end_scores = outputs.end_logits

  start_index = torch.argmax(start_scores)
  end_index = torch.argmax(end_scores)
  # # end_index가 start_index 보다 작거나 같으면 조정
  if end_index == start_index:
    return "찾는 답이 없습니다."

  answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index+1]))
  # 불필요한 공백 제거
  return answer.strip()

In [53]:
# 질문과 문맥
context = '''인공지능은  인간의 학습능력, 추론능력, 지각능력을 인공적으로 구현하려는 컴퓨터 과학의 세부분야 중 하나이다. 정보공학 분야에 있어 하나의 인프라 기술이기도 하다.[1][2] 인간을 포함한 동물이 갖고 있는 지능 즉, 자연 지능(natural intelligence)과는 다른 개념이다.
인간의 지능을 모방한 기능을 갖춘 컴퓨터 시스템이며, 인간의 지능을 기계 등에 인공적으로 시연(구현)한 것이다. 일반적으로 범용 컴퓨터에 적용한다고 가정한다. 이 용어는 또한 그와 같은 지능을 만들 수 있는 방법론이나 실현 가능성 등을 연구하는 과학 기술 분야를 지칭하기도 한다 .
대표적인 인공지능 모델은  초거대 언어 모델(LLM), 적대적 생성 신경망(GAN), 잠재 변수 모델(Latent Variable Models) 등이 있습니다. '''
question = '대표적인 인공지능 모델은 무엇인가요?'
# 질문-답변 생성
answer = answer_question(question,context)
# 결과 출력
print(f"Question: {question}\nAnswer: {answer}")

Question: 대표적인 인공지능 모델은 무엇인가요?
Answer: 초거대 언어 모델 ( LLM )


In [63]:
context = '''
파란색으로 표시된 영역이 우리나라 북쪽으로 올라갑니다.
일주일가량 장기간 한반도를 메웠던 북극 한기가 점차 물러나는 겁니다.
연일 영하 10도를 밑돌았던 서울의 아침 기온은 주말과 비교하면 4도가량 올랐습니다.
전국적으로 내려져 있던 한파특보는 남부지방부터 점차 해제돼 현재는 중부와 경북북부, 전북동부에만 발효 중입니다.
경보와 주의보가 내려진 곳에선 아침까지도 영하 10도 안팎으로 날이 매우 춥겠지만, 낮부터는 이번 한파가 완전히 누그러지겠습니다.
<남민지 기상청 예보분석관> "내륙을 중심으로 아침기온 영하 10도 이하의 강추위가 나타나는 곳이 많겠으나, 낮부터는 따뜻한 남서류가 유입되며 평년수준의 기온으로 회복하겠습니다"
온화한 날씨 속에 정월 대보름날에는 전국 곳곳으로 다시 또 비나 눈이 내릴 전망입니다.
중부지방으로 최고 8㎝에 달하는 눈이 내려 쌓이겠고, 최근 대설이 집중된 전북에서도 1에서 5㎝의 적설이 예상됩니다.
특히, 서해상에서 강한 눈비 구름이 다가오면서 수도권과 강원에선 수요일 출근길에 시간당 1에서 3㎝에 달하는 눈보라도 쏟아지겠습니다.
기상청은 눈비와 함께 전국적인 강풍이 몰아치겠다며, 바깥에 설치된 시설물을 미리 점검하고 약한 구조물의 경우 단단히 고정해 둘 것을 당부했습니다.
'''
question = '온화한 날씨는 언제?'
question2 = '서울의 아침기온은 어떻게 되는가?'
question3 = '언제 눈비가 오는지?'
question4 = '아침 기온은 얼마야?'
# 질문-답변 생성
answer = answer_question(question4,context)
# 결과 출력
print(f"Question: {question4}\nAnswer: {answer}")

Question: 아침 기온은 얼마야?
Answer: 4도
